In [ ]:
%matplotlib ipympl

import fastmap as fm
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import obspy
import sklearn.model_selection
import sklearn.preprocessing
import sklearn.svm

# Build a test database

In [ ]:
nobj = 256

stream = obspy.read("../data/CI.CLC.HHZ.2019.185.msd")

noise = stream.slice(
    starttime=obspy.UTCDateTime("2019-07-04T18:06:10"),
    endtime=obspy.UTCDateTime("2019-07-04T18:06:18")
).normalize()[0]

signal = stream.slice(
    starttime=obspy.UTCDateTime("2019-07-04T18:06:18"),
    endtime=obspy.UTCDateTime("2019-07-04T18:06:26")
).normalize()[0]

database = np.empty((nobj, 801))
labels = list()
for i in range(database.shape[0]):
    label = np.random.choice([0, 1])
    trace = [signal, noise][label].copy()
    trace.data += 2 * np.random.rand() * np.random.choice(noise.data, size=trace.stats.npts)
    trace.normalize()
    database[i, :] = trace.data
    labels.append(label)

# Embed the test database

In [ ]:
fastmap = fm.FastMap(database, fm.distance, 2, "test.hdf5")
fastmap.embed_database();

In [ ]:
plt.close("all")
fig, axes = plt.subplots(nrows=fastmap.ndim, ncols=fastmap.ndim)

for icol in range(fastmap.ndim):
    for irow in range(icol, fastmap.ndim):
        axes[irow, icol].scatter(
            fastmap.image[:, irow],
            fastmap.image[:, icol],
            c=labels,
            cmap=plt.get_cmap("RdBu"),
            s=4
        )

# Perform SVM classification

In [ ]:
X = sklearn.preprocessing.StandardScaler().fit_transform(fastmap.image[:])
y = labels

classifier = sklearn.svm.SVC(gamma=2, C=1)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=1/4)
classifier.fit(X_train, y_train)
classifier.score(X_test, y_test)

In [ ]:
h = 1/32

plt.close("all")
fig, ax = plt.subplots(1, 1)

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

cm = plt.cm.RdBu
cm_bright = mpl.colors.ListedColormap(['#FF0000', '#0000FF'])
# Plot the training points
ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
           edgecolors='k')
# Plot the testing points
ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6,
           edgecolors='k')

Z = classifier.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)